In order to show a simple version of calculating flux and Eiso, we provide a few variables specific to this GRB (Fermi trigger bn080916009) that would either be read in from a file or calculated based on those variables. 

In [1]:
from __future__ import division
import numpy as np
from numpy import exp, power, log, log10, exp

from scipy import integrate
from math import pi


def BAND(energy, alpha, beta, enterm, norm, entype='E0'):
    '''
    BAND(energy, alpha, beta, enterm, norm, entype='E0')

    XSPEC Band Function.

    PARAMETERS:
    ----------
    energy: float, energy element from an array of energies over which 
            the function is integrated over. 
    alpha:  float, alpha (low-energy index) parameter value.  
    beta:   float, beta (high-energy index) parameter value.     
    enterm: float, characteristic energy parameter value.  
    norm:   float, normalization (aka amplitude) parameter value.  
    entype: str, defines which characteristic energy you are using.  
                options: 'E0' (default), 'epeak', 'ebreak'  
    
    '''
    eng     = energy
    N       = norm
    a       = alpha
    b       = beta
    
    if entype == 'epeak':
        t   = enterm / (2.+a)  # E0 = epeak / (2.+alpha)
    elif entype == 'E0':
        t   = enterm           # E0 = E0
    elif entype == 'ebreak':
        t   = enterm/(a-b)     # E0 = ebreak / (alpha - beta)
    else:
        raise Exception, "entype must be 'epeak', 'E0', or 'ebreak' "

    cond1 = eng < (a-b) * t
    cond2 = eng >= (a-b) * t
    
    band = np.piecewise(eng, [cond1, cond2],\
            [lambda x: N * (power(x/100., a) * exp(-x/t)), \
             lambda x: N * (((((a-b)*t)/100.0)**(a-b)) * (exp(b-a))*((x/100.0)**b))])
    return band


In [2]:
def Calc_Flux(model, Pars, emin, emax, redshift):
        
    keVtoerg  = 1.60217657E-9
    emin      = emin/(1.+redshift)
    emax      = emax/(1.+redshift)
    P         = Pars

    function = lambda energy: energy * BAND(energy,  
                                                P['grbm']['alpha'], 
                                                P['grbm']['beta'], 
                                                P['grbm']['enterm'], 
                                                P['grbm']['norm'], 
                                                entype=P['grbm']['entype']
                                                )
    nrgFlux = integrate.quad(function, emin, emax, limit=100)[0] * keVtoerg
    return nrgFlux


In [3]:
# model is the band function, aka 'grbm' for grb model
model = 'grbm'
Pars = {'grbm':         {'alpha':   -1.029586,
                         'beta':    -2.198223,
                         'enterm':  510.641,     # epeak energy
                         'norm':    0.017522454,
                         'entype':  'epeak'}}

# integrate from 10 keV - 10,000 keV
emin     = 10.0
emax     = 10000.0
redshift = 4.35 
# redshift needed to calculate what 10 keV and 10,000 keV are in the rest-frame of the GRB.
# 10 keV/(1+redshift) and 10000 keV/(1+redshift)

# constants needed for eiso calculation
DL       = 1.25078798197959E+29  # GRB's luminosity distance in cm
duration = 62.977 # duration of GRB in seconds

### Flux should be:    1.504698397590E-06

In [4]:
import time  # time the calculation

In [5]:
startTime = time.time()
flux = Calc_Flux(model=model, Pars=Pars, emin=emin, emax=emax, redshift=redshift)
endTime = time.time()

print('Flux:  %.11E  erg cm^-2 s^-1'%flux)
print('Total Time:  %.5f  seconds'%(endTime-startTime))

eiso = ((4.0 * pi * (DL**2))/(1.+redshift)) * (flux * duration)
print('Eiso:  %.11E erg'%eiso)


Flux:  1.50469839759E-06  erg cm^-2 s^-1
Total Time:  0.04673  seconds
Eiso:  3.48220960014E+54 erg
